<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/STT%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [207]:
!pip install tfds-nightly
!pip install pydub

In [208]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [209]:
!git clone https://github.com/Jaidon-Smith/public_datasets.git

fatal: destination path 'public_datasets' already exists and is not an empty directory.


# Normalising all of the audio data with map

[TensorFlow Dataset API](https://www.tensorflow.org/api_docs/python/tf/data/Dataset?version=nightly)

## Demonstrating doing this with mnist

In [210]:
dataset2 = tfds.load("mnist", split="train")

In [211]:
dataset2

<PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>

In [212]:
dataset2 = dataset2.map(lambda x: {"image": x["image"]*0, "label": x["label"]})

In [213]:
dataset2

<MapDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>

## With JSUT

In [214]:
import public_datasets.jsut

In [215]:
download_config = tfds.download.DownloadConfig(manual_dir='/gdrive/MyDrive/datasets/public_datasets/downloads/manual')

dataset = tfds.load("jsut", split="train", data_dir='/gdrive/MyDrive/datasets/public_datasets', download_and_prepare_kwargs={"download_config": download_config})

In [216]:
dataset

<PrefetchDataset shapes: {id: (), speech: (None,), text: ()}, types: {id: tf.string, speech: tf.int64, text: tf.string}>

In [217]:
#dataset = dataset.map(lambda x: {"id": x["id"], "speech": x["speech"]/max(x["speech"]), "text": x["text"]})

In [218]:
dataset = dataset.map(lambda x: {"id": x["id"], "speech": tf.cast(x["speech"]/tf.int16.max, tf.float32), "text": x["text"]})

In [219]:
dataset

<MapDataset shapes: {id: (), speech: (None,), text: ()}, types: {id: tf.string, speech: tf.float32, text: tf.string}>

# Listen to Audio Files in TensorBoard

In [220]:
from datetime import datetime

In [221]:
dataset_4 = dataset.take(4)


In [222]:
audio_4 = list(dataset_4.as_numpy_iterator())

In [223]:
import numpy as np

In [224]:
speech_4 = [x["speech"] for x in audio_4]

In [225]:
speech_4

[array([0.00646992, 0.00653096, 0.006592  , ..., 0.00253304, 0.00253304,
        0.00253304], dtype=float32),
 array([-0.00317392, -0.00311289, -0.00317392, ...,  0.00311289,
         0.00317392,  0.00317392], dtype=float32),
 array([0.00076296, 0.00079348, 0.000824  , ..., 0.00210578, 0.00207526,
        0.00201422], dtype=float32),
 array([-0.00067141, -0.00051881, -0.0003357 , ...,  0.00881985,
         0.00881985,  0.00885037], dtype=float32)]

In [226]:
audio = speech_4[0]
audio = audio.reshape((1, -1, 1))

In [227]:
audio.shape

(1, 458400, 1)

In [228]:
audio = audio.reshape((1, -1, 1))

In [229]:
audio.shape

(1, 458400, 1)

In [230]:
transcripts = [x["text"].numpy().decode('utf-8') for x in dataset_4]

In [231]:
transcripts

['したがって、記事作成に堪える、第三者による評価が明記された資料が、複数発見された場合は、本依頼を取り下げる用意はあります。',
 '仰向けは寝椅子か床、あるいは直に畳に横たわれるので、初心者が練習するのに最も楽な姿勢である。',
 'がまんするのが難しいのは、彼の度を超した丁重さである。',
 '野菜が苦手な子供でも、調味料を工夫すればどんな料理も喜んでパクパク食べてくれるようになる。']

In [232]:
speech_4

[array([0.00646992, 0.00653096, 0.006592  , ..., 0.00253304, 0.00253304,
        0.00253304], dtype=float32),
 array([-0.00317392, -0.00311289, -0.00317392, ...,  0.00311289,
         0.00317392,  0.00317392], dtype=float32),
 array([0.00076296, 0.00079348, 0.000824  , ..., 0.00210578, 0.00207526,
        0.00201422], dtype=float32),
 array([-0.00067141, -0.00051881, -0.0003357 , ...,  0.00881985,
         0.00881985,  0.00885037], dtype=float32)]

In [233]:
description = ''
description += '**Transcript 1**: {}\n\n'.format(transcripts[0])
description += '**Transcript 2**: {}\n\n'.format(transcripts[1])
description += '**Transcript 3**: {}\n\n'.format(transcripts[2])
description += '**Transcript 4**: {}\n\n'.format(transcripts[3])

In [234]:
description

'**Transcript 1**: したがって、記事作成に堪える、第三者による評価が明記された資料が、複数発見された場合は、本依頼を取り下げる用意はあります。\n\n**Transcript 2**: 仰向けは寝椅子か床、あるいは直に畳に横たわれるので、初心者が練習するのに最も楽な姿勢である。\n\n**Transcript 3**: がまんするのが難しいのは、彼の度を超した丁重さである。\n\n**Transcript 4**: 野菜が苦手な子供でも、調味料を工夫すればどんな料理も喜んでパクパク食べてくれるようになる。\n\n'

**Title**

In [235]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

for i in range(4):
  print(i)
  audio = speech_4[i]
  audio = audio.reshape((1, -1, 1))
  description = '**Transcript**: {}\n\n'.format(transcripts[i])
  with file_writer.as_default():
    tf.summary.audio("Audio " + str(i+1), audio, 48000, step=0, description=description)

0
1
2
3


In [236]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [237]:
%tensorboard --logdir logs/train_data

Reusing TensorBoard on port 6006 (pid 557), started 3:22:48 ago. (Use '!kill 557' to kill it.)

<IPython.core.display.Javascript object>

# Encoding the dataset to have 24k sample rate 

## As supervised makes the dataset a tuple rather than dictionary

In [238]:

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [239]:
ds_train

<_OptionsDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [240]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [241]:
ds_train

<_OptionsDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>

## With JSUT


[https://www.tensorflow.org/io/api_docs/python/tfio/audio/resample](https://www.tensorflow.org/io/api_docs/python/tfio/audio/resample)

[https://www.tensorflow.org/io/tutorials/audio](https://www.tensorflow.org/io/tutorials/audio)

In [242]:
!pip install -q tensorflow-io

In [243]:
import tensorflow_io as tfio

In [244]:
import public_datasets.jsut 

In [245]:
download_config = tfds.download.DownloadConfig(manual_dir='/gdrive/MyDrive/datasets/public_datasets/downloads/manual')

dataset, info = tfds.load(
                    "jsut",
                    split="train",
                    data_dir='/gdrive/MyDrive/datasets/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)

In [246]:
dataset

<PrefetchDataset shapes: {id: (), speech: (None,), text: ()}, types: {id: tf.string, speech: tf.int64, text: tf.string}>

In [247]:
#dataset.map(tfio.audio.resample())
#dataset = dataset.map(lambda x: {"id": x["id"], "speech": tf.cast(x["speech"]/tf.int16.max, tf.float32), "text": x["text"]})
dataset = dataset.map(lambda x: {"id": x["id"], "speech": tfio.audio.resample(tf.cast(x["speech"]/tf.int16.max, tf.float32), 48000, 24000), "text": x["text"]})

In [248]:
data = dataset.take(1)
for i in data:
  audio = i
  break


In [249]:
audio_tensor = audio["speech"]

In [250]:
from IPython.display import Audio

Audio(audio_tensor.numpy(), rate=24000)

# Short Time Fourier Transform

[https://www.tensorflow.org/api_docs/python/tf/signal/stft](https://www.tensorflow.org/api_docs/python/tf/signal/stft)

[https://au.mathworks.com/help/signal/ref/stft.html](https://au.mathworks.com/help/signal/ref/stft.html)

In [251]:
print(audio_tensor.shape[0])
print(audio_tensor.shape[0]/24000)

229200
9.55


In [252]:
spectrogram = tf.signal.stft(
      audio_tensor, frame_length=255, frame_step=128)

In [253]:
spectrogram.shape

TensorShape([1789, 129])

In [254]:
spectrogram = tf.signal.stft(
      audio_tensor, frame_length=511, frame_step=256)

In [255]:
spectrogram.shape

TensorShape([894, 257])

In [256]:
spectrogram = tf.signal.stft(
      audio_tensor, frame_length=1023, frame_step=512)
spectrogram.shape

TensorShape([446, 513])

In [257]:
spectrogram = tf.signal.stft(
      audio_tensor, frame_length=2047, frame_step=1024)
spectrogram.shape

TensorShape([222, 1025])

# Understanding 1D Convolutions

In [258]:
model = tf.keras.models.Sequential([
  
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

# Getting the text lengths

In [259]:
!pip install --quiet tensorflow-text

import tensorflow_text as text
from tensorflow.python.platform import gfile

model_file = '/gdrive/MyDrive/Japanese/Bible/letters_v2.model'
model = gfile.GFile(model_file, 'rb').read()

tokenizer = text.SentencepieceTokenizer(model=model)

input_vocab_size = tokenizer.vocab_size().numpy()
print(input_vocab_size)

13306


In [260]:
def preprocess_text(old_text):
  text = old_text
  #text = old_text.numpy()
  #text = 
  #text = text.decode('utf-8')
  text = tokenizer.tokenize(text)
  text = tf.boolean_mask(text, text != 13305)
  #text = text.to_tensor()
  #text = tf.constant(text)
  #text = tf.pad(text, paddings=[[0 , 2000 - tf.shape(text)[0]]])
  return text

In [261]:
def preprocess_audio(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)
  audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  #audio = tf.pad(audio, paddings=[[0 , 2000 - tf.shape(audio)[0]], [0,0]])
  return audio

In [262]:
download_config = tfds.download.DownloadConfig(manual_dir='/gdrive/MyDrive/datasets/public_datasets/downloads/manual')

dataset, info = tfds.load(
                    "jsut",
                    split="train",
                    data_dir='/gdrive/MyDrive/datasets/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)

In [263]:
dataset = dataset.map(lambda x: {"id": x["id"], "speech": preprocess_audio(x["speech"]), "text": preprocess_text(x["text"])})

In [264]:
dataset = dataset.batch(1)



```python
jsut_text_lengths = {}

for i, data in enumerate(dataset):
  
  jsut_text_lengths[data['id'].numpy()[0].decode('utf-8')] = data['text'].shape[1]
  #print(data['id'].numpy()[0].decode('utf-8'), data['text'].shape[1])
  if (i+1)%100 == 0:
    #pass
    print(data['id'].numpy()[0].decode('utf-8'), data['text'].shape[1])
    #break
```

```python
jsut_speech_lengths = {}

for i, data in enumerate(dataset):
  
  jsut_speech_lengths[data['id'].numpy()[0].decode('utf-8')] = data['speech'].shape[1]
  #print(data['id'].numpy()[0].decode('utf-8'), data['text'].shape[1])
  if (i)%100 == 0:
    #pass
    print(data['id'].numpy()[0].decode('utf-8'), data['speech'].shape[1])
    #break
```

```python
# Save these to google drive
import pickle
pickle.dump(jsut_speech_lengths, open('/gdrive/MyDrive/datasets/jsut_speech_lengths', "wb+"))
pickle.dump(jsut_text_lengths, open('/gdrive/MyDrive/datasets/jsut_text_lengths', "wb+"))
```



In [265]:
import pickle
jsut_speech_lengths = pickle.load(open('/gdrive/MyDrive/datasets/jsut_speech_lengths', "rb"))
jsut_text_lengths = pickle.load(open('/gdrive/MyDrive/datasets/jsut_text_lengths', "rb"))

# Building the Model

[This tutorial](https://www.tensorflow.org/tutorials/audio/simple_audio) has some good resources.

In [307]:
import pickle
jsut_speech_lengths = pickle.load(open('/gdrive/MyDrive/datasets/jsut_speech_lengths', "rb"))
jsut_text_lengths = pickle.load(open('/gdrive/MyDrive/datasets/jsut_text_lengths', "rb"))

In [308]:
tokenizer.tokenize(tf.constant("英語が分かりますか"))

<tf.Tensor: shape=(10,), dtype=int32, numpy=
array([13305,  1383,   231,    16,   100,    21,    23,    12,    17,
          21], dtype=int32)>

In [309]:
#def preprocess_audio(old_audio):
#  audio = old_audio/tf.int16.max
#  audio = tf.cast(audio, tf.float32)
#  audio = tfio.audio.resample(audio, 48000, 24000)
#  audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
#  return audio

In [310]:
def preprocess_text(old_text):
  text = old_text
  #text = old_text.numpy()
  #text = 
  #text = text.decode('utf-8')
  text = tokenizer.tokenize(text)
  text = tf.boolean_mask(text, text != 13305)
  #text = text.to_tensor()
  #text = tf.constant(text)
  text = tf.pad(text, paddings=[[0 , 200 - tf.shape(text)[0]]])
  return text

In [311]:
def preprocess_text_no_pad(old_text):
  text = old_text
  #text = old_text.numpy()
  #text = 
  #text = text.decode('utf-8')
  text = tokenizer.tokenize(text)
  text = tf.boolean_mask(text, text != 13305)
  #text = text.to_tensor()
  #text = tf.constant(text)
  #text = tf.pad(text, paddings=[[0 , 200 - tf.shape(text)[0]]])
  text = tf.shape(text)[0]
  return text

In [312]:
# For loop that I don't think will work
#def preprocess_text(old_text):
#  batch_size = 
#  output = 
#  text = old_text.numpy().decode('utf-8')
#  text = tokenizer.tokenize(text)
#  text = tf.constant(text)
#  text = tf.pad(text, paddings=[[0 , 2000 - tf.shape(text)[0]]])
#  return text

In [313]:
#def preprocess_text_batches(old_text):
#  return tf.map_fn(
#    preprocess_text, old_text
#)

In [314]:
#text.numpy()[0].decode('utf-8')

In [315]:
#dataset = dataset.map(lambda x: {"id": x["id"], "speech": preprocess_audio(x["speech"]), "text": x["text"]})

In [316]:
#jsut_speech_lengths = {}

#for i, data in enumerate(dataset):
  
#  jsut_speech_lengths[data['id'].ref()] = data['speech'].shape[0]
#  break

In [317]:
def preprocess_audio(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)

  #audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)

  audio = tf.pad(audio, paddings=[[0 , 500 - tf.shape(audio)[0]], [0,0]])
  return audio

In [318]:
def preprocess_audio_no_pad(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)

  
  #audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)

  #audio = tf.pad(audio, paddings=[[0 , 2000 - tf.shape(audio)[0]], [0,0]])
  audio = tf.shape(audio)[0]
  return audio



```python
def preprocess_audio_no_pad(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)
  audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  #audio = tf.pad(audio, paddings=[[0 , 2000 - tf.shape(audio)[0]], [0,0]])
  speech_lengths = tf.map_fn(lambda x: tf.shape(x)[0], audio, fn_output_signature=tf.int32)
  return speech_lengths
```



In [319]:
download_config = tfds.download.DownloadConfig(manual_dir='/gdrive/MyDrive/datasets/public_datasets/downloads/manual')

dataset, info = tfds.load(
                    "jsut",
                    split="train",
                    data_dir='/gdrive/MyDrive/datasets/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)



```python
dataset = dataset.map(lambda x: {"id": x["id"],
                                 "speech_lengths": tf.py_function(func=preprocess_audio_no_pad, inp=[x["speech"]],Tout=tf.int32),
                                 "speech": preprocess_audio(x["speech"]),
                                 "text": preprocess_text(x["text"])})
```



In [320]:
dataset = dataset.map(lambda x: {"id": x["id"],
                                 "speech": preprocess_audio(x["speech"]),
                                 "speech_lengths": preprocess_audio_no_pad(x["speech"]),
                                 "text": preprocess_text(x["text"]),
                                 "text_lengths": preprocess_text_no_pad(x["text"])})

In [321]:
#dataset = dataset.map(lambda x: {"id": x["id"], "speech": preprocess_audio(x["speech"]), "text": x["text"]})

In [322]:
dataset = dataset.batch(8)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [323]:
#max = 0

#for i, data in enumerate(dataset):
#  value = data['speech'].shape[0]
#  if value > max:
#    max = value
#    print("new max is:", value, "with id:", data['id'].numpy())

In [324]:
ds = dataset.take(1)

In [325]:
for i in ds:
  data = i
  break

In [326]:
print(data["speech_lengths"])

tf.Tensor([222 183  99 169  69 246 100 108], shape=(8,), dtype=int32)


In [327]:
print(tf.shape(data['text']).numpy())

[  8 200]


In [328]:
print(data['text'][0][:100])

tf.Tensor(
[   5    3   16   25   11    2  255  174  335  516    7 1900   35   15
    2  366  191   53    7   34   15 1055  788   16  313  255   26   18
    3 1012  758   16    2 1873  389  642   89   26   18    3  239  219
    9    2  493 1623  406    8  149   23  187   77   15  331  248    9
   27   23   12   17    4    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0], shape=(100,), dtype=int32)


In [329]:
#input_vocab_size = tokenizer.vocab_size() - 1
input_vocab_size = 2000
print(input_vocab_size)

tf.Tensor(13305, shape=(), dtype=int32)


In [330]:
speech = data['speech']
text = data['text']
id = data['id']
speech_lengths = data['speech_lengths']
text_lengths = data['text_lengths']

In [331]:
text_lengths

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([61, 46, 27, 45, 18, 69, 25, 26], dtype=int32)>

In [332]:
speech_lengths

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([222, 183,  99, 169,  69, 246, 100, 108], dtype=int32)>

# Using Transformer Encoder Blocks

In [333]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [334]:
def scaled_dot_product_attention(q, k, v):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)


  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [335]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

In [336]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [337]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training):

    attn_output, _ = self.mha(x, x, x)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [338]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.embedding = tf.keras.layers.Dense(d_model)

    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # Makes input larger by a constant
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)



```python
model = tf.keras.models.Sequential([
                                    
  Encoder(num_layers=6,
          d_model=512,
          num_heads=8,
          dff=2048, 
          pe_input=10000),

  Dense(input_vocal_size)
])
```



## CTC Loss



```python
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')
```



In [339]:
#dense1 = tf.keras.layers.Dense(input_vocab_size)

In [340]:
dense1 = tf.keras.layers.Dense(2000)

In [341]:
dense1_speech = dense1(speech)

In [342]:
dense1_speech.shape

TensorShape([8, 500, 2000])



```python
text_lengths = tf.map_fn(lambda x: jsut_text_lengths[x.numpy().decode('utf-8')], id, fn_output_signature=tf.int32)

speech_lengths = tf.map_fn(lambda x: jsut_speech_lengths[x.numpy().decode('utf-8')], id, fn_output_signature=tf.int32)
```



In [343]:
labels = text
logits = dense1_speech
label_length = text_lengths
logit_length = speech_lengths
unique = tf.nn.ctc_unique_labels(text)


loss = tf.nn.ctc_loss(
    labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
)

In [346]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 226196278679775354, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14838578260371540432
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]